# Initialization

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import optuna
import joblib

from typing import List
import string

In [2]:
RANDOM_STATE = 0
N_JOBS = 6

In [3]:
import sys
import os

current_folder = os.path.abspath('') 

notebooks_folder = os.path.dirname(current_folder)

project_root = os.path.dirname(notebooks_folder)

if project_root not in sys.path:
    sys.path.append(project_root)

# Reading data

In [4]:
train_df = pd.read_csv("../../data/processed/train_language_detection_dataset.csv")
train_df.head()

,sentence,lan_code
0,"Merry Christmas, Tatoeba!",eng
1,"Крайне важно, чтобы мы поговорили с Томом.",rus
2,Она была весела.,rus
3,Urban sprawl is said to be a major contributor...,eng
4,Только не надо делать большие глаза.,rus


In [5]:
final_test_df = pd.read_csv("../../data/processed/test_language_detection_dataset.csv")
final_test_df.head()

,sentence,lan_code
0,She suspected that it was too late.,eng
1,Я порой бываю рассеян.,rus
2,Он мог бы победить.,rus
3,"Том мог быть не таким счастливым, каким прикид...",rus
4,"Do not forsake me, oh my darling.",eng


In [6]:
X_train = train_df["sentence"]
y_train = train_df["lan_code"]

In [7]:
X_final_test = final_test_df["sentence"]
y_final_test = final_test_df["lan_code"]

# Encoders

In this section we'll compare the learning speed of several different encoders for our texts as well as prepare different preprocessing approaches.

We should also note that all of our encoders are gonna tokenize text by characters and not by words for the reason that is we're trying to classify text and not to find some deep semantic meaning of it, for which separation by characters is a much better approach (it can handle typos and only "looks" for the character distribution by which language can be determined pretty accurately, but we still may be caring for characters order and for this reason we're gonna use custom n-grams parameter values).

## Preprocessors

In this section we're gonna write some simple different preprocessors.

In [7]:
def simple_preprocessor(text: str) -> str:
    return text

In [8]:
def no_punctuation_preprocessor(text: str) -> str:
    blacklist_chars = string.punctuation
    translation_table = str.maketrans("", "", blacklist_chars)
    cleaned_text = text.translate(translation_table)
    return cleaned_text

In [9]:
preprocessors = {
    "simple": simple_preprocessor,
    "no_punct": no_punctuation_preprocessor
}

We're also gonna copy our preprocessor functions as a custom model to the file located at `../../app/models/transformers.py` in order to be able to use them from anywhere later without need to redefine them (and we also may need this for our final model to work properly).

## TF-IDF

scikin-learn's implementation of TF-IDF is offline, which means that it keeps everything in-memory and should be trained in one run. It may not be viable for training and tuning our models, but we still gonna check the perfomance of this approach as well.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
tfidf = TfidfVectorizer(
    analyzer='char_wb', # We're caring only about characters in word bounds
    ngram_range=(1,3),
    max_features=100000,
    preprocessor=no_punctuation_preprocessor,
    lowercase=True,
)

Note that we're not using stopwords as they can be a crutial part in differentiating between languages from one language family (exactly Russian and Ukrainian).

In [15]:
tfidf.fit(X_train)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,<function no_...0021314003BA0>
,tokenizer,None
,analyzer,'char_wb'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


As we can see, this TF-IDF implementation has been training for a pretty long time considering that we'll be tuning hyperparameters of both encoder and our classification model, so we're not likely to use it considering that it is also an offline implementation.

## HashingVectorizer

HashingVectorizer is an online approach, which means that we can partially train it. Also, its scikit-learn implementation should be noticably faster than the implementation of TF-IDF (although it's resuls can be quite worse, but it's still faster to train which means that it's also faster to tune).

In [16]:
from sklearn.feature_extraction.text import HashingVectorizer

In [17]:
vectorizer = HashingVectorizer(
    analyzer='char_wb',
    ngram_range=(1,3),
    n_features=2**20,
    lowercase=True,
    preprocessor=no_punctuation_preprocessor
)

In [20]:
vectorizer.fit_transform(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 172011046 stored elements and shape (2273614, 1048576)>

As this vectorizer is online, it's fit is stateless, but it's whole expensive computation is performed during transform stage. In result we can see, that it's still faster than TF-IDF, so we'll use it during our hyperparameters tuning.

# Models

We'll try three different models - SGDClassifier, Multinomial NaiveBayes and XGBoost, all of which are pretty popular in text classification tasks.

In [8]:
from sklearn.feature_extraction.text import HashingVectorizer

In [9]:
from app.models.transformers import TextCleaner

## SGDClassifier

In [10]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [11]:
sgd_model_path = "../../models/dev/sgd_language_detection_model.joblib"

### Tuning

In [14]:
def sgdclassifier_objective(trial: optuna.Trial):
    # TextCleaner hyperparameters
    text_cleaner_mode = trial.suggest_categorical("text_cleaner_mode", ["simple", "no_punct"])

    text_cleaner = TextCleaner(
        mode=text_cleaner_mode
    )

    # HashingVectorizer hyperparameters
    n_features_pow = trial.suggest_int("n_features_pow", 12, 20)
    ngram_max = trial.suggest_int("ngram_max", 3, 5)

    vectorizer = HashingVectorizer(
        analyzer='char_wb',
        ngram_range=(1, ngram_max),
        n_features=2 ** n_features_pow,
        alternate_sign=True,
        norm='l2',
        lowercase=True
    )

    # SGDClassifier hyperparameters
    alpha = trial.suggest_float("alpha", 1e-6, 1e-2, log=True)
    penalty = trial.suggest_categorical("penalty", ["l2", "elasticnet"])

    # ElasticNet mixing parameter, will be ignored by model if penalty is l2
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)

    clf = SGDClassifier(
        loss='log_loss', # LogisticRegression
        penalty=penalty,
        alpha=alpha,
        l1_ratio=l1_ratio,
        class_weight="balanced", # Crutial because we have big class imbalance in our training dataset
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS
    )

    pipeline = Pipeline([
        ("text_cleaner", text_cleaner),
        ("vectorizer", vectorizer),
        ("classifier", clf)
    ])

    scores = cross_val_score(
        pipeline,
        X_train,
        y_train,
        cv=3,
        scoring="f1_macro", # Because of our class imbalance
        n_jobs=N_JOBS
    )

    return scores.mean()

Note that we skipped max_iter parameter because it does not impact partial_fit behaviour according to https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html.

In [15]:
study = optuna.create_study(direction="maximize")
study.optimize(sgdclassifier_objective, n_trials=20, show_progress_bar=True)

[I 2026-01-06 13:43:46,778] A new study created in memory with name: no-name-c5388e98-9e9b-4efa-9263-8c4150b205fe


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2026-01-06 13:47:09,585] Trial 0 finished with value: 0.7825068112260452 and parameters: {'text_cleaner_mode': 'no_punct', 'n_features_pow': 13, 'ngram_max': 3, 'alpha': 0.0032034087882173833, 'penalty': 'elasticnet', 'l1_ratio': 0.9272854236095921}. Best is trial 0 with value: 0.7825068112260452.
[I 2026-01-06 13:50:25,820] Trial 1 finished with value: 0.6490575977714391 and parameters: {'text_cleaner_mode': 'simple', 'n_features_pow': 18, 'ngram_max': 3, 'alpha': 0.007978455193278595, 'penalty': 'elasticnet', 'l1_ratio': 0.6309709514070638}. Best is trial 0 with value: 0.7825068112260452.
[I 2026-01-06 13:53:11,336] Trial 2 finished with value: 0.7791140345527499 and parameters: {'text_cleaner_mode': 'no_punct', 'n_features_pow': 20, 'ngram_max': 3, 'alpha': 0.0022769798693948007, 'penalty': 'l2', 'l1_ratio': 0.48065368689884713}. Best is trial 0 with value: 0.7825068112260452.
[I 2026-01-06 13:56:18,171] Trial 3 finished with value: 0.9845540476698081 and parameters: {'text_clean

In [16]:
study.best_params

{'text_cleaner_mode': 'simple',
 'n_features_pow': 15,
 'ngram_max': 4,
 'alpha': 1.1179323625363607e-06,
 'penalty': 'elasticnet',
 'l1_ratio': 0.4518974786326927}

```
{'text_cleaner_mode': 'simple',
 'n_features_pow': 15,
 'ngram_max': 4,
 'alpha': 1.1179323625363607e-06,
 'penalty': 'elasticnet',
 'l1_ratio': 0.4518974786326927}
```

### Saving model

We'll save best parameters into a separate variable in order to not need to go through hyperparameter tuning once again to get them.

In [17]:
sgd_best_params = {
    'text_cleaner_mode': 'simple',
    'n_features_pow': 15,
    'ngram_max': 4,
    'alpha': 1.1179323625363607e-06,
    'penalty': 'elasticnet',
    'l1_ratio': 0.4518974786326927
}

Now we're gonna save our model using joblib in order to not go through hyperparameter tuning stage again in the future.

As our text cleaner parameter ended up being `simple`, our model will lead best perfomance without additional preprocessing applied on text.

In [18]:
try:
    params = study.best_params
except:
    params = sgd_best_params

final_sgd_vectorizer = HashingVectorizer(
    analyzer='char_wb',
    ngram_range=(1, params["ngram_max"]),
    n_features=2 ** params["n_features_pow"],
    alternate_sign=True,
    norm='l2',
    lowercase=True
)

final_sgd_clf = SGDClassifier(
    loss='log_loss',
    penalty=params["penalty"],
    alpha=params["alpha"],
    l1_ratio=params["l1_ratio"],
    class_weight="balanced",
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS
)

final_sgd_pipeline = Pipeline([
    ("vectorizer", final_sgd_vectorizer),
    ("classifier", final_sgd_clf)
])

In [19]:
final_sgd_pipeline.fit(X_train, y_train)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [20]:
joblib.dump(final_sgd_pipeline, sgd_model_path)

['../../models/dev/sgd_language_detection_model.joblib']

### Loading model

And now we can load our already tuned model and see it's structure.

In [21]:
sgd_model = joblib.load(sgd_model_path)
sgd_model

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


As we've stated earlier, we can use our model as-is - no additional text preprocessing needed!

## MultinomialNB

In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [23]:
mnb_model_path = "../../models/dev/mnb_language_detection_model.joblib"

### Tuning

In [24]:
def sgdclassifier_objective(trial: optuna.Trial):
    # TextCleaner hyperparameters
    text_cleaner_mode = trial.suggest_categorical("text_cleaner_mode", ["simple", "no_punct"])

    text_cleaner = TextCleaner(
        mode=text_cleaner_mode
    )

    # HashingVectorizer hyperparameters
    n_features_pow = trial.suggest_int("n_features_pow", 12, 20)
    ngram_max = trial.suggest_int("ngram_max", 3, 5)
    vectorizer_norm = trial.suggest_categorical("vectorizer_norm", ["l2", None])

    vectorizer = HashingVectorizer(
        analyzer='char_wb',
        ngram_range=(1, ngram_max),
        n_features=2 ** n_features_pow,
        alternate_sign=False, # Values must be non-negative for MultinomialNB
        norm=vectorizer_norm,
        lowercase=True
    )

    # MultinomialNB hyperparameters
    alpha = trial.suggest_float("alpha", 1e-10, 10.0, log=True)
    fit_prior = trial.suggest_categorical("fit_prior", [True, False])

    clf = MultinomialNB(
        alpha=alpha,
        fit_prior=fit_prior
    )

    pipeline = Pipeline([
        ("text_cleaner", text_cleaner),
        ("vectorizer", vectorizer),
        ("classifier", clf)
    ])

    scores = cross_val_score(
        pipeline,
        X_train,
        y_train,
        cv=3,
        scoring="f1_macro", # Because of our class imbalance
        n_jobs=N_JOBS
    )

    return scores.mean()

In [25]:
study = optuna.create_study(direction="maximize")
study.optimize(sgdclassifier_objective, n_trials=20, show_progress_bar=True)

[I 2026-01-06 15:04:56,770] A new study created in memory with name: no-name-c04703a1-67e4-4848-982d-bb1e7add2106


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2026-01-06 15:07:20,542] Trial 0 finished with value: 0.9895692914602493 and parameters: {'text_cleaner_mode': 'simple', 'n_features_pow': 20, 'ngram_max': 3, 'vectorizer_norm': 'l2', 'alpha': 0.0019069600154671383, 'fit_prior': False}. Best is trial 0 with value: 0.9895692914602493.
[I 2026-01-06 15:09:53,948] Trial 1 finished with value: 0.9828492028356766 and parameters: {'text_cleaner_mode': 'no_punct', 'n_features_pow': 12, 'ngram_max': 3, 'vectorizer_norm': 'l2', 'alpha': 9.044882026469729e-10, 'fit_prior': False}. Best is trial 0 with value: 0.9895692914602493.
[I 2026-01-06 15:12:48,900] Trial 2 finished with value: 0.9936525177018432 and parameters: {'text_cleaner_mode': 'no_punct', 'n_features_pow': 16, 'ngram_max': 4, 'vectorizer_norm': 'l2', 'alpha': 2.6870705572275184e-09, 'fit_prior': False}. Best is trial 2 with value: 0.9936525177018432.
[I 2026-01-06 15:16:09,882] Trial 3 finished with value: 0.9889555688910273 and parameters: {'text_cleaner_mode': 'no_punct', 'n_fe

In [26]:
study.best_params

{'text_cleaner_mode': 'simple',
 'n_features_pow': 19,
 'ngram_max': 5,
 'vectorizer_norm': None,
 'alpha': 0.00022553046850033962,
 'fit_prior': True}

```
{'text_cleaner_mode': 'simple',
 'n_features_pow': 19,
 'ngram_max': 5,
 'vectorizer_norm': None,
 'alpha': 0.00022553046850033962,
 'fit_prior': True}
 ```

### Saving model

We'll save best parameters to a separate variable in order to not need to go through hyperparameter tuning once again to get them.

In [27]:
mnb_best_params = {
    'text_cleaner_mode': 'simple',
    'n_features_pow': 19,
    'ngram_max': 5,
    'vectorizer_norm': None,
    'alpha': 0.00022553046850033962,
    'fit_prior': True
}

Now we're gonna save our model using joblib in order to not go through hyperparameter tuning stage again in the future.

As our text cleaner parameter ended up being `simple`, our model will lead best perfomance without additional preprocessing applied on text.

In [28]:
try:
    params = study.best_params
except:
    params = mnb_best_params

final_mnb_vectorizer = HashingVectorizer(
    analyzer='char_wb',
    ngram_range=(1, params["ngram_max"]),
    n_features=2 ** params["n_features_pow"],
    alternate_sign=False,
    norm=params["vectorizer_norm"],
    lowercase=True
)

final_mnb_clf = MultinomialNB(
    alpha=params["alpha"],
    fit_prior=params["fit_prior"]
)

final_mnb_pipeline = Pipeline([
    ("vectorizer", final_mnb_vectorizer),
    ("classifier", final_mnb_clf)
])

In [29]:
final_mnb_pipeline.fit(X_train, y_train)

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [30]:
joblib.dump(final_mnb_pipeline, mnb_model_path)

['../../models/dev/mnb_language_detection_model.joblib']

### Loading model

And now we can load our already tuned model and see it's structure.

In [31]:
mnb_model = joblib.load(mnb_model_path)
mnb_model

,steps,"[('vectorizer', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


As we've stated earlier, we can use our model as-is - no additional text preprocessing needed!